In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix , auc, precision_recall_curve, roc_auc_score
from scipy.stats import uniform, randint

In [ ]:
dataset_gold = pd.read_csv('../../assets/gold/dataset_gold_v1.csv')
dataset_gold.drop(columns = ['dni', 'nacionalidad', 'tiene_beca'], inplace=True)
dataset_gold.drop(columns = ['situacion_padre', 'situacion_madre'], inplace=True)
dataset_gold.drop(columns=['edad', 'cantidad_de_veces_que_rindio_cluster_8.0', 'cursada_por_uba_xxi_cluster_8.0', 'nota_cluster_8.0', 'fecha_promedio_cluster_8.0'], inplace=True)

In [ ]:
dataset_gold['sexo'].value_counts()

In [ ]:
dataset_gold['sexo'] = dataset_gold['sexo'].replace({'Masculino': 0, 'Femenino': 1})
dataset_gold['sexo'].value_counts()

In [ ]:
dataset_gold['cohorte'].value_counts(dropna=False)

In [ ]:
dataset_gold['cohorte'] = dataset_gold['cohorte'].replace({'2021C1': 0, '2021C2': 1,
                                                           '2022C1': 2, '2022C2': 3})
dataset_gold['cohorte'].value_counts(dropna=False)

In [ ]:
dataset_gold['carrera'].unique()

In [ ]:
dataset_gold['carrera'].value_counts(dropna=False)

In [ ]:
# Frecuentes
label_map = {
    'Licenciatura en Ciencias de la Computación': 0,
    'Licenciatura en Ciencias Biológicas': 1,
    'Licenciatura en Ciencias Físicas': 2,
    'Licenciatura en Ciencias de Datos': 3,
    'Licenciatura en Ciencias Químicas': 4,
    'Licenciatura en Ciencias Matemáticas': 5,
    'Licenciatura en Ciencias de la Atmósfera': 6,
    'Licenciatura en Ciencia y Tecnología de Alimentos': 7,
    'Licenciatura en Ciencias Geológicas': 8,
    
    # Agrupadas todas bajo el mismo número
    'Materias de Otras Carreras': 9,
    'Profesorado de Enseñanza Media y Superior en Matemática': 9,
    'Licenciatura en Paleontología': 9,
    'Licenciatura en Ciencias Oceanográficas': 9,
    'Profesorado de Enseñanza Media y Superior en la Especialidad Biología': 9,
    'Profesorado de Enseñanza Media y Superior en Ciencias de la Computación': 9,
    'Profesorado de Enseñanza Media y Superior en Física': 9,
    'Profesorado de Enseñanza Media y Superior en Química': 9,
    'Propuesta Materias Niveladoras de Postgrado': 9
}

# Aplicar el encoding directamente
dataset_gold['carrera_encoded'] = dataset_gold['carrera'].map(label_map)
dataset_gold['carrera_encoded'].value_counts(dropna=False)

In [ ]:
dataset_gold.drop(columns=['carrera'], inplace=True)

In [ ]:
dataset_gold['nivel_estudio_padre'].value_counts(dropna=False)

In [ ]:
dataset_gold['nivel_estudio_madre'].value_counts(dropna=False)

In [ ]:
nivel_educativo_map = {
    'Est. de Post grado': 0,
    'Est. universitarios completos': 1,
    'Est. universitarios incompletos': 2,
    'Est. superiores completos': 4,
    'Est. superiores incompletos': 5,
    'Colegio Secundario completo': 6,
    'Colegio Secundario incompleto': 7,
    'Escuela primaria completa': 8,
    'Escuela primaria incompleta': 9,
    'Desconoce': 10,
    'No hizo estudios': 11
}

# Aplicar el encoding directamente
dataset_gold['nivel_estudio_madre_encoded'] = dataset_gold['nivel_estudio_madre'].map(nivel_educativo_map)
dataset_gold['nivel_estudio_madre_encoded'].value_counts(dropna=False)

In [ ]:
# Aplicar el encoding directamente
dataset_gold['nivel_estudio_padre_encoded'] = dataset_gold['nivel_estudio_padre'].map(nivel_educativo_map)
dataset_gold['nivel_estudio_padre_encoded'].value_counts(dropna=False)

In [ ]:
dataset_gold.drop(columns=['nivel_estudio_padre', 'nivel_estudio_madre'], inplace=True)

In [ ]:
dataset_gold['situacion_laboral'].value_counts(dropna=False)

In [ ]:
situacion_laboral_map = {
    'Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)': 0,
    'No trabajó y buscó trabajo en algún momento de los últimos 30 días': 1,
    'No trabajó y no buscó trabajo (no esta pensando en trabajar)': 2,
    'Desconoce': 3
}
# Aplicar el encoding directamente
dataset_gold['situacion_laboral_encoded'] = dataset_gold['situacion_laboral'].map(situacion_laboral_map)
dataset_gold['situacion_laboral_encoded'].value_counts(dropna=False)

In [ ]:
dataset_gold.drop(columns=['situacion_laboral'], inplace=True)

In [ ]:
for i in range(1, 7):
    # Mapear los valores booleanos a enteros, manteniendo los nulos
    dataset_gold[f'cursada_por_uba_xxi_cluster_{i}.0'] = dataset_gold[f'cursada_por_uba_xxi_cluster_{i}.0'].map({True: 1, False: 0})

    # Convertir la columna al tipo Int64 que permite nulos
    dataset_gold[f'cursada_por_uba_xxi_cluster_{i}.0'] = dataset_gold[f'cursada_por_uba_xxi_cluster_{i}.0'].astype('Int64')

    print(dataset_gold[f'cursada_por_uba_xxi_cluster_{i}.0'].value_counts(dropna=False))

In [ ]:
print(dataset_gold.dtypes.to_markdown())

In [ ]:
print(dataset_gold.isnull().sum().to_markdown())

## Dividimos los datos

En realidad estaría dividiendo en entrenamiento y validación, pues no tenemos el conjunto de test final acá.

In [ ]:
dataset_gold.to_csv('../../assets/gold/experimentos_finales/dataset_final.csv', index=False)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
                                    dataset_gold[dataset_gold.columns.difference(['target'])],
                                    dataset_gold['target'],
                                    random_state=5, #original lo hice con 4, 49 y 50 dan buen importancia pero muy malas metricas, 51 y 55 no está tan mal
                                    test_size=0.2,
                                    stratify=dataset_gold['target'])

In [ ]:
print("Proporción de datos del dataset de original: \n " , pd.DataFrame(dataset_gold["target"].value_counts(normalize=True)))
print("Proporción de datos del dataset de evaluación: \n ", pd.DataFrame(y_test.value_counts(normalize=True)))
print("Proporción de datos del dataset de entrenamiento: \n ", pd.DataFrame(y_train.value_counts(normalize=True)))

In [ ]:
x_test.to_csv('../../assets/gold/experimentos_finales_2/x_test.csv', index=False)
y_test.to_csv('../../assets/gold/experimentos_finales_2/y_test.csv', index=False)
x_train.to_csv('../../assets/gold/experimentos_finales_2/x_train.csv', index=False)
y_train.to_csv('../../assets/gold/experimentos_finales_2/y_train.csv', index=False)